<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH18_01_colab_TPU_%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 코랩(Colab)에서 TPU 선택

### 2. TPU 초기화
-----------------
딥러닝 모델을 정의하기 전에 아래의 설정을 미리 해주어야 하므로 아래의 코드는 초반부에 실행해 줘야 함

In [2]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])


tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

### 3. TPU Strategy 셋팅
--------------------------
tf.distribute.Strategy는 훈련을 여러 GPU 또는 장비, TPU로 나누어 처리하기 위한 텐서플로 API임. 이 API를 사용하면 기존의 모델이나 훈련코드를 분산처리할 수 있음. TPU 사용을 위해서도 Strategy를 셋팅해 줌

In [3]:
strategy = tf.distribute.TPUStrategy(resolver)

### 4. 딥러닝 모델의 컴파일
------------------------
딥러닝 모델을 컴파일 할때도 추가적인 코드가 필요함. 모델의 컴파일은 strategy.scope 내에서 이루어져야 함. 이는 모델의 층을 쌓는 create_model() 와 같은 함수를 만들어 strategy.scope 내에서 해당함수를 호출하여 모델을 컴파일하는 방식으로 할 수 있음. 예를 들어....


다음과 같이 모델의 층을 쌓는 create_model()라는 함수를 만듦

In [4]:
def create_model():
  return tf.keras.Sequential([
      tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(256, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)])

In [5]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam', 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=['sparse_categorical_accuracy'])

이 모델을 fit() 하게 되면 해당 모델은 TPU를 사용하여 학습하게 됨. 위의 코드를 모두 실행하여 BERT와 커스텀 레이어에 대한 모델의 컴파일을 strategy.scope 내에서 진행함


### 5. GPU 실습 코드로 되돌리기
---------------------
이번 챕터에서 TPU로 실습을 진행한 코드들을 GPU 환경에서 실습하고 싶다면 위에서 모델의 층을 쌓고, 컴파일 하는 코드를 제외한 TPU 진행만을 위한 코드들을 전부 제외하면 됨